In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer


In [2]:
#R1764, R1762

# Read Files

In [3]:
pd.set_option('display.max_columns', None)

#Set to view all characters in column(default is 50, replace None with 50 once done viewing)
pd.set_option('display.max_colwidth', None)

faults_orig = pd.read_csv('../data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp', 'LocationTimeStamp']) #1187335

#Make a copy to refer
faults = faults_orig
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intake NOx,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


## 2 - Vehicle Onboard Data


In [4]:
vdod_orig = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
vdod = vdod_orig

# Create a wider table from long table, and drop the Name 
vdod = vdod.pivot(index="FaultId", columns="Name", values="Value").reset_index()
vdod



Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [5]:
vdod.columns

Index(['FaultId', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object', name='Name')

In [6]:
# Remove commas from all 11 columns
for col in vdod.columns[:21]:
    vdod[col] = vdod[col].astype(str).str.replace(',', '')

# Convert all columns to numeric
vdod = vdod.apply(pd.to_numeric, errors='coerce')

vdod

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,NaN,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.00,1632.20,43.2,12300.907429,0.000000,NaN,NaN,78.8,1023,NaN,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,NaN,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.25,10722.70,96.4,58979.184416,7.647805,32.0,NaN,98.6,18431,NaN,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,NaN,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.75,9326.75,100.0,65080.105870,8.995086,NaN,NaN,91.4,17407,NaN,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,NaN,67.72946,28606.65625,181.4,0.0,27.26,221.73120,863.25,586.75,23.6,4042.492826,0.000000,NaN,NaN,100.4,1023,NaN,NaN,11.84489,14.10,100.0,1.74


In [7]:
vdod = vdod.drop(columns=['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])
vdod

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.00,1632.20,43.2,12300.907429,0.000000,NaN,78.8,1023,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.25,10722.70,96.4,58979.184416,7.647805,32.0,98.6,18431,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.75,9326.75,100.0,65080.105870,8.995086,NaN,91.4,17407,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.73120,863.25,586.75,23.6,4042.492826,0.000000,NaN,100.4,1023,NaN,11.84489,14.10,100.0,1.74


### 3 Service Fault Codes


In [8]:
fc = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')
fc

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failure - Bad Intelligent Device or Component,Error internal to the ECM related to memory hardware failures or internal ECM voltage supply circuits.
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechanical System Not Responding or Out of Adjustment,Mechanical failure in the engine timing actuator driver circuit.
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltage Above Normal or Shorted to High Source,High signal voltage detected at the engine timing actuator driver circuit
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltage Below Normal or Shorted to Low Source,Low voltage detected at the engine timing actuator driver circuit.
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,"Engine Magnetic Speed/Position Lost Both of Two Signals - Data Erratic, Intermittent, or Incorrect",The ECM has detected that the primary and backup speed sensor signals are connected backwards.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


# Data Prep

In [ ]:
#Merge faults and vehicle dignostic tables
#faults = pd.merge(faults, vdod, left_on='RecordID', right_on='FaultId')



In [9]:
faults # 1187335 rows × 45 columns

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intake NOx,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


## Clean up faults data


In [10]:
#Remove faults where the EquipmentID has more than 5 characters.
faults = (
    faults.drop(['actionDescription', 'faultValue'], axis=1)
    [faults['EquipmentID'].str.len() <= 5]
)

In [11]:
for lat, lon in [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]:    
    faults = faults.loc[~((abs(lat - faults['Latitude']) <= 0.01) &
                          (abs(lon - faults['Longitude']) <= 0.01))]

In [12]:
faults = faults.loc[faults['EventTimeStamp'].dt.year > 2011]

In [13]:
#remove the rows where active = False
faults = faults.loc[faults['active'] == True]

In [14]:
#Create new column EventTimeStamp_DateOnly with only date part
faults['EventTimeStamp_DateOnly']= faults['EventTimeStamp'].dt.date 
faults['LocationTimeStamp_DateOnly']= faults['LocationTimeStamp'].dt.date 

faults['EventTimeStamp_DateOnly'] = pd.to_datetime(faults['EventTimeStamp_DateOnly'])
faults['LocationTimeStamp_DateOnly'] = pd.to_datetime(faults['LocationTimeStamp_DateOnly'])

In [15]:
#Filter out 'R1762', 'R1764'
faults = faults[~faults[['EquipmentID']].isin(['R1762', 'R1764', '2185A']).any(axis=1)]


In [16]:
faults = faults.astype({'EquipmentID': 'int'})

In [17]:
#faults = faults.astype({'EquipmentID': 'int'})
faults.sort_values(by='EquipmentID',ascending=False)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly
1187298,1248422,123883295,2020-03-06 11:56:38,Low (Severity Medium) Transmission Air Tank Pressure,5516018*202.56.0*5516502*E003.e003*5539634*86.21*5538103*4*,Z0098155,EEO-xxF112C,EATON,3,37,18,True,2,2381,108616503,33.612638,-84.384444,2020-03-06 11:57:53,2020-03-06,2020-03-06
1186588,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,True,1,2380,108616776,35.491620,-86.434074,2020-03-04 10:06:47,2020-03-04,2020-03-04
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06
1185425,1246549,122833965,2020-02-28 04:56:18,Low (Severity Medium) Engine Coolant Level,NaN,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,4,2377,108605700,35.426157,-84.688657,2020-02-28 04:57:05,2020-02-28,2020-02-28
1186237,1247361,123313030,2020-03-03 07:47:01,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,5,2377,108605700,38.404675,-78.912546,2020-03-03 07:47:37,2020-03-03,2020-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152445,1209669,107700945,2019-10-26 09:57:59,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,63,301,105441522,36.131712,-86.401111,2019-10-26 09:58:34,2019-10-26,2019-10-26
1152435,1209659,107697310,2019-10-26 09:14:05,Error in System Engine Exhaust Gas Recirculation (EGR) Valve 2 Control,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,3821,11,True,47,301,105441522,36.450138,-86.466203,2019-10-26 09:16:34,2019-10-26,2019-10-26
1152434,1209658,107697303,2019-10-26 09:13:45,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,49,301,105441522,36.454722,-86.466574,2019-10-26 09:16:14,2019-10-26,2019-10-26
1152427,1209651,107693969,2019-10-26 08:35:57,Error in System Engine Exhaust Gas Recirculation (EGR) Valve 2 Control,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,3821,11,True,36,301,105441522,36.861944,-86.453425,2019-10-26 08:36:33,2019-10-26,2019-10-26


In [ ]:
#faults[(faults['EquipmentID']== '1499') & (faults['spn'] == 5246)]

In [18]:
#View the new date columns 
faults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546664 entries, 0 to 1187333
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   RecordID                    546664 non-null  int64         
 1   ESS_Id                      546664 non-null  int64         
 2   EventTimeStamp              546664 non-null  datetime64[ns]
 3   eventDescription            517005 non-null  object        
 4   ecuSoftwareVersion          414242 non-null  object        
 5   ecuSerialNumber             393170 non-null  object        
 6   ecuModel                    518561 non-null  object        
 7   ecuMake                     518561 non-null  object        
 8   ecuSource                   546664 non-null  int64         
 9   spn                         546664 non-null  int64         
 10  fmi                         546664 non-null  int64         
 11  active                      546664 non

# EDA

In [ ]:
all_trucks = faults['EquipmentID'].unique()
partial_derate = faults.loc[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].unique()
total_derate = faults.loc[faults['spn'] == 5246]['EquipmentID'].unique()

partial_derate_only = partial_derate[np.isin(partial_derate, total_derate, invert=True)]
total_derate_only = total_derate[np.isin(total_derate, partial_derate, invert=True)]
partial_and_total_derate = np.intersect1d(partial_derate, total_derate)
no_derate = all_trucks[np.isin(all_trucks, partial_derate_only, invert=True) | np.isin(all_trucks, total_derate_only, invert=True)]

In [ ]:
print("partial_derate_only = " + str(len(partial_derate_only)))
print("total_derate_only = " + str(len(total_derate_only)))
print("partial_and_total_derate = " + str(len(partial_and_total_derate)))
print("no_derate = " + str(len(no_derate)))

## Find vehicle EquipmentID for which the derate has happened

In [ ]:
#df_faults =faults[faults['EquipmentID'].isin(uEQ)]
#faults[(faults['EquipmentID']== 2381)]

In [19]:
df_faults = faults

In [ ]:
#df_faults['EventTimeStamp_DateOnly'] = pd.to_datetime(df_faults['EventTimeStamp_DateOnly'])
df_faults= (
    faults.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','RecordID' ], ascending = [False, False, False])
)
df_faults.head(30)

#eyeball the dataframe
#df_faults.to_csv('df_faults.csv')  

### There are some spn which occur at the same time in faults DF, e.g. in below 5246 spn is in middle even though the eventTimestamp is same. So to capture the first 5 rows
### Create a dummy column 'DummyDerate_5246', DummyDerate_1569 and assign value 1 for the respecive spn

#### 6285	305	9/26/2018 12:37	1060723	9/26/2018	TRUE	1761	19	1
#### 6284	305	9/26/2018 12:37	1060722	9/26/2018	TRUE	74	14	1
#### 6283	305	9/26/2018 12:37	1060721	9/26/2018	TRUE	632	3	1
#### 6282	305	9/26/2018 12:37	1060720	9/26/2018	TRUE	171	2	1
#### 6281	305	9/26/2018 12:37	1060719	9/26/2018	TRUE	3697	9	1
### 6280	305	9/26/2018 12:37	1060718	9/26/2018	TRUE	5246	19	1
#### 6279	305	9/26/2018 12:37	1060717	9/26/2018	TRUE	1072	5	1
#### 6278	305	9/26/2018 12:37	1060716	9/26/2018	TRUE	1231	2	1
#### 6277	305	9/26/2018 12:37	1060715	9/26/2018	TRUE	1668	2	1
#### 6276	305	9/26/2018 12:37	1060714	9/26/2018	TRUE	3821	11	1
#### 6275	305	9/26/2018 12:37	1060713	9/26/2018	TRUE	1761	14	1
#### 6274	305	9/26/2018 12:37	1060711	9/26/2018	TRUE	171	3	1


In [20]:
df_faults['DummyDerate_5246'] = np.where(df_faults['spn'] == 5246, 1, 0)
df_faults['DummyDerate_1569'] = np.where(df_faults['spn'] == 1569, 1, 0)

In [21]:
# Check df
print(df_faults[df_faults['DummyDerate_5246'] == 1])
print(df_faults[df_faults['DummyDerate_1569'] == 1])

         RecordID     ESS_Id      EventTimeStamp eventDescription  \
2089         2090    1011009 2015-02-23 05:05:44              NaN   
2971         2972    1026305 2015-02-23 15:54:22              NaN   
5713         5714    1070646 2015-02-25 13:53:08              NaN   
6534         6535    1097942 2015-02-26 22:24:29              NaN   
6628         6629    1104361 2015-02-27 09:09:56              NaN   
...           ...        ...                 ...              ...   
1173036   1232200  116794344 2020-01-13 13:18:31              NaN   
1178571   1238712  119571469 2020-02-03 15:46:46              NaN   
1179414   1239555  119959276 2020-02-06 07:45:08              NaN   
1181700   1241841  120905759 2020-02-13 13:32:39              NaN   
1181996   1242137  121038018 2020-02-14 11:21:54              NaN   

                                      ecuSoftwareVersion ecuSerialNumber  \
2089     05290170*03015749*051914190353*09400015*G1*BDR*        79642446   
2971               

In [22]:
df_faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25,2015-02-21,2015-02-21,0,0
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10,2015-02-21,2015-02-21,0,0
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08,2015-02-21,2015-02-21,0,0
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59,2015-02-21,2015-02-21,0,0
6,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29,2015-02-21,2015-02-21,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187324,1248448,123899434,2020-03-06 13:12:43,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1936,105355619,30.376851,-81.744953,2020-03-06 13:29:33,2020-03-06,2020-03-06,0,0
1187328,1248452,123901805,2020-03-06 13:42:48,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,True,93,1886,105351219,39.015694,-77.031157,2020-03-06 13:43:24,2020-03-06,2020-03-06,0,0
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59,2020-03-06,2020-03-06,0,1
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intake NOx,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14,2020-03-06,2020-03-06,0,0


## Sort df_faults in descending order and save df_faults to csv to use in subsequent steps

In [ ]:

df_faults_5246= (
    df_faults.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi', 'DummyDerate_5246'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','DummyDerate_5246','RecordID' ], ascending = [False, False, False, False])
)

df_faults_5246.head

#df_faults_5246[(df_faults_5246['RecordID']< 1060735) & (df_faults_5246['EquipmentID'] == 305) ].head(30)

#df_faults_5246[(df_faults_5246['spn']  == 5246) ]

#eyeball the dataframe and save it. 
#df_faults_5246.to_csv('df_faults_5246.csv')  

In [ ]:
df_faults_1569= (
    df_faults.groupby(['EquipmentID','EventTimeStamp','RecordID' ,'EventTimeStamp_DateOnly','active', 'spn', 'fmi', 'DummyDerate_1569'])
    .size().reset_index(name='count')
    .sort_values(by=['EquipmentID','EventTimeStamp','DummyDerate_1569','RecordID' ], ascending = [False, False, False, False])
)

#df_faults_1569
df_faults_1569[(df_faults_1569['spn']  == 1569) ]

#eyeball the dataframe and save it. 
#df_faults_1569.to_csv('df_faults_1569.csv')  

In [99]:
df_faults_1569

NameError: name 'df_faults_1569' is not defined

# function to filter faults dataframe.
### This will filter the data by looking up SPN=1569 and fmi= 31, and then will traverse in by picking the spn which occured prio with in the date window specified


In [ ]:
#When calling this function for spn 5246 or 1569. Update the if condition below.
# I will modify the code to handle it automatically later
def  GetFilteredSPNbyDays(df_faults, windowTimeframeUnit, day_window):
    df_new = pd.DataFrame(columns = ['RecordID','EquipmentID', 'EventTimeStamp','EventTimeStamp_DateOnly','active', 'spn', 'fmi','Derate', 'group','target'])  
    #print(df_new)
    #df_new = df_new.astype({'EquipmentID': 'int'})
    dts_evt = ""
    dts_evt_max=""

    hasDerate = False
    counter =0
    derateGroupCounter = 0
    # loop through rows of original dataframe and assign new values to columns of new dataframe
    for index, row in df_faults.iterrows():
        counter+=1       
        #print('derateGroupCounter== ' + str(derateGroupCounter))
        #if((row['spn'] == 1569) & (row['fmi'] == 31)): 
        #print('-1 - . EquipmentID = ' + str(row['EquipmentID']))
        if((row['spn'] == 5246)): 
        #if((row['spn'] == 1569) & (row['fmi'] == 31)): 
            #print('0. spn Check Counter = ' + str(counter))          
            #print('0 - A. Counter = ' + str(hasDerate))
            derateGroupCounter +=1
            
            hasDerate = True
            df_new.loc[index, 'Derate'] = pd.to_numeric(1)
            df_new.loc[index, 'target'] = 0
            dts_evt = row['EventTimeStamp'] 
            #dts_evt_max = dts_evt - timedelta(days=day_window)
            if(windowTimeframeUnit == "hours"):                   
                dts_evt_max = dts_evt - timedelta(hours=day_window)
                #print("hours = dts_evt_max= " + str(dts_evt_max))
            elif(windowTimeframeUnit == "days"):                   
                dts_evt_max = dts_evt - timedelta(days=day_window)
                #print("days = dts_evt_max= " + str(dts_evt_max))
            #add cols
            #print('1. Counter = ' + str(counter))
            #print('1. RecordID = ' + str(row['RecordID']))
            #print('dts_evt =' + str(dts_evt))
            #print('dts_evt_max =' + str(dts_evt_max))
           
            df_new.loc[index, 'RecordID'] = row['RecordID']
            df_new.loc[index, 'EquipmentID'] = pd.to_numeric(row['EquipmentID']) 
            df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
            df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
            df_new.loc[index, 'active'] = row['active']
            df_new.loc[index, 'spn'] = row['spn']
            df_new.loc[index, 'fmi'] = row['fmi']
            df_new.loc[index, 'group'] = derateGroupCounter
        else:
            #print('18. Counter = ' + str(counter))
            if(dts_evt_max != ""):
                #print('20. Counter = ' + str(counter))
                #print('21. RecordID = ' + str(row['RecordID']))
                #print('22 In Else after dts_evt_max != ' + str((row['EventTimeStamp'] > dts_evt_max)))
                #print((row['EventTimeStamp'] > dts_evt_max))
                if((row['EventTimeStamp'] > dts_evt_max) & (hasDerate)) :
                    #print('EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    #print('23 EventTimeStamp > dts_evt_max' + str(dts_evt_max) + " ---- " + str(row['EventTimeStamp_DateOnly']))
                    df_new.loc[index, 'Derate'] = pd.to_numeric(0)
                    df_new.loc[index, 'target'] = 1
                    df_new.loc[index, 'RecordID'] = row['RecordID']
                    df_new.loc[index, 'EquipmentID'] = pd.to_numeric(row['EquipmentID']) 
                    df_new.loc[index, 'EventTimeStamp'] = row['EventTimeStamp']
                    df_new.loc[index, 'EventTimeStamp_DateOnly'] = row['EventTimeStamp_DateOnly'] 
                    df_new.loc[index, 'active'] = row['active']
                    df_new.loc[index, 'spn'] = row['spn']
                    df_new.loc[index, 'fmi'] = row['fmi']
                    df_new.loc[index, 'group'] = derateGroupCounter
                    #print('3. Counter = ' + str(counter))
                else:
                    #print('ELSE  ' + str(dts_evt_max)+ " ---- " +  str(row['EventTimeStamp_DateOnly']))
                    hasDerate = False
                    dts_evt = ""
                    dts_evt_max = ""
                    #print('3. Counter else else = ' + str(counter))
    #print(df_new) 
    
    return df_new


In [ ]:
# Test by passing 1000 rows and 3 days lookup timeframe
#dfW= GetFilteredSPNbyDays(df_faults.head(11000), 'hours',6)
#print(dfW) 


In [ ]:
#df4= faults.merge(dfW, on='RecordID', how='left', indicator=True)
#df4

In [ ]:
#get the filtered SPN  5246 for 6 hours timeframe
df_filtered_5246 = pd.DataFrame()
df_filtered_5246.info()
chunkcount = 0
for chunk in pd.read_csv("df_faults_5246.csv", chunksize=20000, parse_dates=['EventTimeStamp']):
    chunkcount +=1
    dfW = GetFilteredSPNbyDays(chunk,"hours", 6)
    print(chunkcount)
    df_filtered_5246 = pd.concat([df_filtered_5246, dfW])
    
#save the dataframe to compare with the one in prior step to visually compare few rows. This csv contains the target variable
df_filtered_5246.to_csv('df_filtered_5246.csv') 





In [ ]:
df_filtered_5246

In [ ]:
#get the filtered SPN  1569 for 6 hours timeframe
df_filtered_1569 = pd.DataFrame()
df_filtered_1569.info()
chunkcount = 0
for chunk in pd.read_csv("df_faults_1569.csv", chunksize=20000, parse_dates=['EventTimeStamp']):
    chunkcount +=1
    dfW = GetFilteredSPNbyDays(chunk,"hours", 6)
    print(chunkcount)
    df_filtered_1569 = pd.concat([df_filtered_1569, dfW])
    
df_filtered_1569
#save the dataframe to compare with the one in prior step 
df_filtered_1569.to_csv('df_filtered_1569.csv')

In [ ]:
df_filtered_1569

In [ ]:
#save the dataframe to compare with the one in prior step. This csv contains the target variable
df_filtered_1569.to_csv('df_filtered_1569.csv')  

In [ ]:
#Merge the filtered SPN with the original faults dataframe
#faults_filtered =faults.merge(dfW, on='RecordID', how='left', indicator=True)

In [ ]:
diagnostics_imputed = ""

### Impute the values


In [23]:
# Michael's function to deal with the empty lists that were causing trouble above...
imputer = SimpleImputer(strategy='mean')

def impute_values(x):
    imputer_results = imputer.fit_transform(x.values.reshape(-1,1))
    
    if len(imputer_results[0]) == 0:
        return np.array([np.nan] * len(x))
    return imputer_results

In [46]:
diagnostics_imputed = faults.merge(vdod, left_on='RecordID', right_on='FaultId')

# this below is needed so that we can simply reassign back to the dataframe
# and that's because if we sort it by EquipmentID, then the grouping and apply is not going to change the order
diagnostics_imputed = diagnostics_imputed.sort_values(by='EquipmentID')

In [47]:
diagnostics_imputed

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
228959,480159,9286652,2016-05-19 09:44:56,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,94,301,105431712,36.000185,-86.596250,2016-05-19 09:45:33,2016-05-19,2016-05-19,0,0,480159,0.0,14.5000,0.00000,218451.500000,174.2,12.0,28.42,200.8062,649.750,4441.55,73.200000,30525.608953,0.673641,109.4,131.0,17407,NaN,0.000000,NaN,0.0,0.29
227657,477361,9225085,2016-05-16 13:55:05,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,91,301,105431712,35.881064,-84.508148,2016-05-16 13:55:41,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.00000,218074.600000,186.8,9.0,22.04,208.9063,649.250,4433.35,45.600000,30476.340865,0.581180,123.8,158.0,17407,NaN,7.854520,NaN,0.0,0.00
474960,1059736,62708616,2018-09-24 14:28:35,Low Voltage (Aftertreatment 1 Particulate Trap Differential Pressure),unknown,unknown,unknown,unknown,0,3251,4,True,8,301,105351510,35.901527,-86.872777,2018-09-24 14:29:12,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.00000,370590.900000,156.2,19.0,31.32,174.8750,653.500,8390.15,76.400000,51537.457711,1.122735,116.6,93.2,17407,NaN,4.757373,NaN,0.0,1.45
474961,1059737,62709570,2018-09-24 14:34:28,Data May Be Invalid J1939 Network #4,unknown,unknown,unknown,unknown,0,1668,19,True,52,301,105351510,35.901527,-86.872777,2018-09-24 14:35:05,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.00000,370591.000000,167.0,0.0,5.80,175.7750,0.000,8390.25,71.600000,51537.457711,0.000000,116.6,93.2,17407,NaN,0.000000,NaN,0.0,0.00
534988,1219096,111152924,2019-11-23 07:30:05,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,1,301,105441522,35.815324,-86.399814,2019-11-23 07:30:42,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.00000,451774.500000,158.0,8.0,30.74,181.1750,652.000,11040.70,62.800000,63434.313986,0.541554,80.6,93.2,17407,NaN,12.733260,NaN,0.0,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540505,1232638,116966923,2020-01-14 13:46:39,Abnormal Frequency J1939 Network #2,unknown,unknown,unknown,unknown,49,1231,8,True,2,2377,108605700,41.937500,-72.231250,2020-01-14 14:23:16,2020-01-14,2020-01-14,0,0,1232638,100.0,14.7900,66.48672,7006.758789,176.0,36.0,40.60,202.3813,1336.625,142.30,8.400001,1004.910486,6.036350,NaN,84.2,50175,NaN,158.447200,13.95,100.0,6.38
543952,1241225,120678988,2020-02-12 05:42:02,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,2,2377,108605700,36.663333,-86.553240,2020-02-12 05:42:38,2020-02-12,2020-02-12,0,0,1241225,6.8,14.3550,64.62260,18709.183594,82.4,25.0,48.14,66.7625,829.000,408.45,66.800000,2662.590112,1.862419,NaN,55.4,2047,NaN,3.126274,13.90,100.0,1.45
546663,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.656250,181.4,0.0,27.26,221.7312,863.250,586.75,23.600000,4042.492826,0.000000,NaN,100.4,1023,NaN,11.844890,14.10,100.0,1.74
546324,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000

In [48]:
cols = ['activeTransitionCount','MCTNumber', 'AcceleratorPedal',
         'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
         'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 
        'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 
        'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 
        'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']


for column in cols:
    # double explode!! can't simply assign back because the right side's EquipmentID is not a unique index
    diagnostics_imputed[column] = diagnostics_imputed.groupby('EquipmentID')[column].apply(lambda x: impute_values(x)).explode().explode().array

    


In [27]:
diagnostics_imputed

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
228959,480159,9286652,2016-05-19 09:44:56,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,94.0,301,105431712.0,36.000185,-86.596250,2016-05-19 09:45:33,2016-05-19,2016-05-19,0,0,480159,0.0,14.5,0.0,218451.5,174.2,12.0,28.42,200.8062,649.75,4441.55,73.2,30525.608953,0.673641,109.4,131.0,17407.0,NaN,0.0,3276.75,0.0,0.29
227657,477361,9225085,2016-05-16 13:55:05,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,91.0,301,105431712.0,35.881064,-84.508148,2016-05-16 13:55:41,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.0,218074.6,186.8,9.0,22.04,208.9063,649.25,4433.35,45.6,30476.340865,0.58118,123.8,158.0,17407.0,NaN,7.85452,3276.75,0.0,0.0
474960,1059736,62708616,2018-09-24 14:28:35,Low Voltage (Aftertreatment 1 Particulate Trap Differential Pressure),unknown,unknown,unknown,unknown,0,3251,4,True,8.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:29:12,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.0,370590.9,156.2,19.0,31.32,174.875,653.5,8390.15,76.4,51537.457711,1.122735,116.6,93.2,17407.0,NaN,4.757373,3276.75,0.0,1.45
474961,1059737,62709570,2018-09-24 14:34:28,Data May Be Invalid J1939 Network #4,unknown,unknown,unknown,unknown,0,1668,19,True,52.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:35:05,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.0,370591.0,167.0,0.0,5.8,175.775,0.0,8390.25,71.6,51537.457711,0.0,116.6,93.2,17407.0,NaN,0.0,3276.75,0.0,0.0
534988,1219096,111152924,2019-11-23 07:30:05,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,1.0,301,105441522.0,35.815324,-86.399814,2019-11-23 07:30:42,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.0,451774.5,158.0,8.0,30.74,181.175,652.0,11040.7,62.8,63434.313986,0.541554,80.6,93.2,17407.0,NaN,12.73326,3276.75,0.0,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540505,1232638,116966923,2020-01-14 13:46:39,Abnormal Frequency J1939 Network #2,unknown,unknown,unknown,unknown,49,1231,8,True,2.0,2377,108605700.0,41.937500,-72.231250,2020-01-14 14:23:16,2020-01-14,2020-01-14,0,0,1232638,100.0,14.79,66.48672,7006.758789,176.0,36.0,40.6,202.3813,1336.625,142.3,8.400001,1004.910486,6.03635,NaN,84.2,50175.0,NaN,158.4472,13.95,100.0,6.38
543952,1241225,120678988,2020-02-12 05:42:02,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,2.0,2377,108605700.0,36.663333,-86.553240,2020-02-12 05:42:38,2020-02-12,2020-02-12,0,0,1241225,6.8,14.355,64.6226,18709.183594,82.4,25.0,48.14,66.7625,829.0,408.45,66.8,2662.590112,1.862419,NaN,55.4,2047.0,NaN,3.126274,13.9,100.0,1.45
546663,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8.0,2377,108605700.0,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,100.4,1023.0,NaN,11.84489,14.1,100.0,1.74
546324,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,True,1.0,2380,108616776.0,35.491620,-86.434074,20

In [28]:
diagnostics_imputed.columns

Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
       'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'EventTimeStamp_DateOnly',
       'LocationTimeStamp_DateOnly', 'DummyDerate_5246', 'DummyDerate_1569',
       'FaultId', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature',
       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IntakeManifoldTemperature', 'LampStatus', 'ServiceDistance', 'Speed',
       'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure'],
      dtype='object')

In [49]:
#Add the dummy target column before merging the filtred spn dataframes for 1569 and 5246
diagnostics_imputed['target'] = 0

In [50]:
df_filtered_5246 =  pd.read_csv('df_filtered_5246.csv', low_memory=False) 


In [52]:
diagnostics_imputed

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
228959,480159,9286652,2016-05-19 09:44:56,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,94.0,301,105431712.0,36.000185,-86.596250,2016-05-19 09:45:33,2016-05-19,2016-05-19,0,0,480159,0.0,14.5,0.0,218451.5,174.2,12.0,28.42,200.8062,649.75,4441.55,73.2,30525.608953,0.673641,109.4,131.0,17407.0,NaN,0.0,3276.75,0.0,0.29,0
227657,477361,9225085,2016-05-16 13:55:05,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,91.0,301,105431712.0,35.881064,-84.508148,2016-05-16 13:55:41,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.0,218074.6,186.8,9.0,22.04,208.9063,649.25,4433.35,45.6,30476.340865,0.58118,123.8,158.0,17407.0,NaN,7.85452,3276.75,0.0,0.0,0
474960,1059736,62708616,2018-09-24 14:28:35,Low Voltage (Aftertreatment 1 Particulate Trap Differential Pressure),unknown,unknown,unknown,unknown,0,3251,4,True,8.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:29:12,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.0,370590.9,156.2,19.0,31.32,174.875,653.5,8390.15,76.4,51537.457711,1.122735,116.6,93.2,17407.0,NaN,4.757373,3276.75,0.0,1.45,0
474961,1059737,62709570,2018-09-24 14:34:28,Data May Be Invalid J1939 Network #4,unknown,unknown,unknown,unknown,0,1668,19,True,52.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:35:05,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.0,370591.0,167.0,0.0,5.8,175.775,0.0,8390.25,71.6,51537.457711,0.0,116.6,93.2,17407.0,NaN,0.0,3276.75,0.0,0.0,0
534988,1219096,111152924,2019-11-23 07:30:05,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,1.0,301,105441522.0,35.815324,-86.399814,2019-11-23 07:30:42,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.0,451774.5,158.0,8.0,30.74,181.175,652.0,11040.7,62.8,63434.313986,0.541554,80.6,93.2,17407.0,NaN,12.73326,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540505,1232638,116966923,2020-01-14 13:46:39,Abnormal Frequency J1939 Network #2,unknown,unknown,unknown,unknown,49,1231,8,True,2.0,2377,108605700.0,41.937500,-72.231250,2020-01-14 14:23:16,2020-01-14,2020-01-14,0,0,1232638,100.0,14.79,66.48672,7006.758789,176.0,36.0,40.6,202.3813,1336.625,142.3,8.400001,1004.910486,6.03635,NaN,84.2,50175.0,NaN,158.4472,13.95,100.0,6.38,0
543952,1241225,120678988,2020-02-12 05:42:02,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,2.0,2377,108605700.0,36.663333,-86.553240,2020-02-12 05:42:38,2020-02-12,2020-02-12,0,0,1241225,6.8,14.355,64.6226,18709.183594,82.4,25.0,48.14,66.7625,829.0,408.45,66.8,2662.590112,1.862419,NaN,55.4,2047.0,NaN,3.126274,13.9,100.0,1.45,0
546663,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8.0,2377,108605700.0,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,100.4,1023.0,NaN,11.84489,14.1,100.0,1.74,0
546324,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,True,1.0,2380,10861677

# Prep Data for full derate 5246

In [37]:
diagnostics_imputed_5246= diagnostics_imputed.merge(df_filtered_5246, left_on='RecordID', right_on='RecordID')

In [56]:
df_filtered_5246.columns

Index(['Unnamed: 0', 'RecordID', 'EquipmentID', 'EventTimeStamp',
       'EventTimeStamp_DateOnly', 'active', 'spn', 'fmi', 'Derate', 'group',
       'target'],
      dtype='object')

In [93]:
faults_all_5246=(
        pd.merge(diagnostics_imputed, df_filtered_5246[['RecordID','target']], left_on='RecordID', right_on='RecordID', how='left')
    )
faults_all_5246

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target_x,target_y
0,480159,9286652,2016-05-19 09:44:56,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,94.0,301,105431712.0,36.000185,-86.596250,2016-05-19 09:45:33,2016-05-19,2016-05-19,0,0,480159,0.0,14.5,0.0,218451.5,174.2,12.0,28.42,200.8062,649.75,4441.55,73.2,30525.608953,0.673641,109.4,131.0,17407.0,NaN,0.0,3276.75,0.0,0.29,0,NaN
1,477361,9225085,2016-05-16 13:55:05,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,91.0,301,105431712.0,35.881064,-84.508148,2016-05-16 13:55:41,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.0,218074.6,186.8,9.0,22.04,208.9063,649.25,4433.35,45.6,30476.340865,0.58118,123.8,158.0,17407.0,NaN,7.85452,3276.75,0.0,0.0,0,NaN
2,1059736,62708616,2018-09-24 14:28:35,Low Voltage (Aftertreatment 1 Particulate Trap Differential Pressure),unknown,unknown,unknown,unknown,0,3251,4,True,8.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:29:12,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.0,370590.9,156.2,19.0,31.32,174.875,653.5,8390.15,76.4,51537.457711,1.122735,116.6,93.2,17407.0,NaN,4.757373,3276.75,0.0,1.45,0,NaN
3,1059737,62709570,2018-09-24 14:34:28,Data May Be Invalid J1939 Network #4,unknown,unknown,unknown,unknown,0,1668,19,True,52.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:35:05,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.0,370591.0,167.0,0.0,5.8,175.775,0.0,8390.25,71.6,51537.457711,0.0,116.6,93.2,17407.0,NaN,0.0,3276.75,0.0,0.0,0,NaN
4,1219096,111152924,2019-11-23 07:30:05,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,1.0,301,105441522.0,35.815324,-86.399814,2019-11-23 07:30:42,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.0,451774.5,158.0,8.0,30.74,181.175,652.0,11040.7,62.8,63434.313986,0.541554,80.6,93.2,17407.0,NaN,12.73326,3276.75,0.0,0.29,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,116966923,2020-01-14 13:46:39,Abnormal Frequency J1939 Network #2,unknown,unknown,unknown,unknown,49,1231,8,True,2.0,2377,108605700.0,41.937500,-72.231250,2020-01-14 14:23:16,2020-01-14,2020-01-14,0,0,1232638,100.0,14.79,66.48672,7006.758789,176.0,36.0,40.6,202.3813,1336.625,142.3,8.400001,1004.910486,6.03635,NaN,84.2,50175.0,NaN,158.4472,13.95,100.0,6.38,0,NaN
546660,1241225,120678988,2020-02-12 05:42:02,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,2.0,2377,108605700.0,36.663333,-86.553240,2020-02-12 05:42:38,2020-02-12,2020-02-12,0,0,1241225,6.8,14.355,64.6226,18709.183594,82.4,25.0,48.14,66.7625,829.0,408.45,66.8,2662.590112,1.862419,NaN,55.4,2047.0,NaN,3.126274,13.9,100.0,1.45,0,NaN
546661,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8.0,2377,108605700.0,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,100.4,1023.0,NaN,11.84489,14.1,100.0,1.74,0,NaN
546662,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,

In [94]:
# merge  target_x, target_y and then dro[ them]
faults_all_5246["target"] = faults_all_5246[['target_x', 'target_y']].any(axis='columns')
faults_all_5246 =  faults_all_5246.drop(columns=['target_x', 'target_y'])
faults_all_5246.target = faults_all_5246.target.replace({True: 1, False: 0})
faults_all_5246

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,480159,9286652,2016-05-19 09:44:56,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,94.0,301,105431712.0,36.000185,-86.596250,2016-05-19 09:45:33,2016-05-19,2016-05-19,0,0,480159,0.0,14.5,0.0,218451.5,174.2,12.0,28.42,200.8062,649.75,4441.55,73.2,30525.608953,0.673641,109.4,131.0,17407.0,NaN,0.0,3276.75,0.0,0.29,0
1,477361,9225085,2016-05-16 13:55:05,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5396,1,True,91.0,301,105431712.0,35.881064,-84.508148,2016-05-16 13:55:41,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.0,218074.6,186.8,9.0,22.04,208.9063,649.25,4433.35,45.6,30476.340865,0.58118,123.8,158.0,17407.0,NaN,7.85452,3276.75,0.0,0.0,0
2,1059736,62708616,2018-09-24 14:28:35,Low Voltage (Aftertreatment 1 Particulate Trap Differential Pressure),unknown,unknown,unknown,unknown,0,3251,4,True,8.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:29:12,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.0,370590.9,156.2,19.0,31.32,174.875,653.5,8390.15,76.4,51537.457711,1.122735,116.6,93.2,17407.0,NaN,4.757373,3276.75,0.0,1.45,0
3,1059737,62709570,2018-09-24 14:34:28,Data May Be Invalid J1939 Network #4,unknown,unknown,unknown,unknown,0,1668,19,True,52.0,301,105351510.0,35.901527,-86.872777,2018-09-24 14:35:05,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.0,370591.0,167.0,0.0,5.8,175.775,0.0,8390.25,71.6,51537.457711,0.0,116.6,93.2,17407.0,NaN,0.0,3276.75,0.0,0.0,0
4,1219096,111152924,2019-11-23 07:30:05,Incorrect Data Engine Exhaust Back Pressure Regulator Solenoid,PC4_21408P151F*,Y044053,MX16U15D13,PCAR,0,649,2,True,1.0,301,105441522.0,35.815324,-86.399814,2019-11-23 07:30:42,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.0,451774.5,158.0,8.0,30.74,181.175,652.0,11040.7,62.8,63434.313986,0.541554,80.6,93.2,17407.0,NaN,12.73326,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,116966923,2020-01-14 13:46:39,Abnormal Frequency J1939 Network #2,unknown,unknown,unknown,unknown,49,1231,8,True,2.0,2377,108605700.0,41.937500,-72.231250,2020-01-14 14:23:16,2020-01-14,2020-01-14,0,0,1232638,100.0,14.79,66.48672,7006.758789,176.0,36.0,40.6,202.3813,1336.625,142.3,8.400001,1004.910486,6.03635,NaN,84.2,50175.0,NaN,158.4472,13.95,100.0,6.38,0
546660,1241225,120678988,2020-02-12 05:42:02,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,80225924,6X1u17D1500000000,CMMNS,0,111,18,True,2.0,2377,108605700.0,36.663333,-86.553240,2020-02-12 05:42:38,2020-02-12,2020-02-12,0,0,1241225,6.8,14.355,64.6226,18709.183594,82.4,25.0,48.14,66.7625,829.0,408.45,66.8,2662.590112,1.862419,NaN,55.4,2047.0,NaN,3.126274,13.9,100.0,1.45,0
546661,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8.0,2377,108605700.0,35.030925,-85.321527,2020-03-06 14:14:49,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,100.4,1023.0,NaN,11.84489,14.1,100.0,1.74,0
546662,1247712,123513793,2020-03-04 10:06:10,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,True,1.0,2380,108616776.0,35.491620,-86.434074,

In [95]:
faults_all_5246.columns


Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
       'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'EventTimeStamp_DateOnly',
       'LocationTimeStamp_DateOnly', 'DummyDerate_5246', 'DummyDerate_1569',
       'FaultId', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature',
       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IntakeManifoldTemperature', 'LampStatus', 'ServiceDistance', 'Speed',
       'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'target'],
      dtype='object')

In [96]:
faults_all_5246.isna().sum()

RecordID                           0
ESS_Id                             0
EventTimeStamp                     0
eventDescription               29659
ecuSoftwareVersion            132422
ecuSerialNumber               153494
ecuModel                       28103
ecuMake                        28103
ecuSource                          0
spn                                0
fmi                                0
active                             0
activeTransitionCount              0
EquipmentID                        0
MCTNumber                          0
Latitude                           0
Longitude                          0
LocationTimeStamp                  0
EventTimeStamp_DateOnly            0
LocationTimeStamp_DateOnly         0
DummyDerate_5246                   0
DummyDerate_1569                   0
FaultId                            0
AcceleratorPedal               24475
BarometricPressure                 0
CruiseControlSetSpeed          11240
DistanceLtd                        0
E

In [97]:
# Delete columns
faults_all_5246 = faults_all_5246.drop(columns=['ESS_Id', 'eventDescription','ecuSoftwareVersion', 'ecuSerialNumber', 
    'ecuModel', 'ecuMake', 'ecuSource', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp', 'ServiceDistance'])
faults_all_5246

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,DummyDerate_5246,DummyDerate_1569,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,0,0,480159,0.0,14.5,0.0,218451.5,174.2,12.0,28.42,200.8062,649.75,4441.55,73.2,30525.608953,0.673641,109.4,131.0,17407.0,NaN,0.0,3276.75,0.0,0.29,0
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,0,0,477361,0.0,14.4275,0.0,218074.6,186.8,9.0,22.04,208.9063,649.25,4433.35,45.6,30476.340865,0.58118,123.8,158.0,17407.0,NaN,7.85452,3276.75,0.0,0.0,0
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,0,0,1059736,0.0,14.4275,0.0,370590.9,156.2,19.0,31.32,174.875,653.5,8390.15,76.4,51537.457711,1.122735,116.6,93.2,17407.0,NaN,4.757373,3276.75,0.0,1.45,0
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,0,0,1059737,0.0,14.4275,0.0,370591.0,167.0,0.0,5.8,175.775,0.0,8390.25,71.6,51537.457711,0.0,116.6,93.2,17407.0,NaN,0.0,3276.75,0.0,0.0,0
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,0,0,1219096,0.0,14.2825,0.0,451774.5,158.0,8.0,30.74,181.175,652.0,11040.7,62.8,63434.313986,0.541554,80.6,93.2,17407.0,NaN,12.73326,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,0,0,1232638,100.0,14.79,66.48672,7006.758789,176.0,36.0,40.6,202.3813,1336.625,142.3,8.400001,1004.910486,6.03635,NaN,84.2,50175.0,NaN,158.4472,13.95,100.0,6.38,0
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,0,0,1241225,6.8,14.355,64.6226,18709.183594,82.4,25.0,48.14,66.7625,829.0,408.45,66.8,2662.590112,1.862419,NaN,55.4,2047.0,NaN,3.126274,13.9,100.0,1.45,0
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,0,0,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,100.4,1023.0,NaN,11.84489,14.1,100.0,1.74,0
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,0,0,1247712,0.0,14.2825,0.0,511.711578,167.0,12.0,30.74,165.65,599.625,22.75,32.0,65.514669,0.607598,NaN,118.4,1023.0,NaN,0.0,13.95,100.0,0.29,0
